In [1]:
file = 'data/bipartite/crime/train.in'
N = 5

# Cross Validation

In [2]:
import cross_validation
cross_validation.cross_validation(file, N, 5)

TypeError: sfdp_layout() got an unexpected keyword argument 'bipartite'

In [ ]:
file = 'data/bipartite/movilens/train.in'
N = 5
cross_validation.cross_validation(file, N, 5)

# Basic Workflow

In [112]:
import tools

In [113]:
train_set, nodes, poss_set, neg_set = tools.sample_bipartite(file, N)

# Choose poss set with less components increasing

In [91]:
from graph_tool.all import *
g = Graph(directed=False)
g.add_vertex(max(nodes) + 1)
for edge in train_set:
    u, w = map(int, edge.split())
    g.add_edge(g.vertex(u), g.vertex(w))

KeyboardInterrupt: 

In [ ]:
train_set_2 = set()
for edge in train_set:
    u, w = map(int, edge.split())
    if g.vertex(u).out_degree() > 1 and g.vertex(w).out_degree() > 1:
        train_set_2.add(edge)

In [48]:
len(train_set_2)

724

In [49]:
poss_set = tools.sample_structural_poss(train_set_2, int(N * len(train_set) / 100.))

# Get coordinates

In [114]:
from graph_tool.all import *
g = Graph(directed=False)
g.add_vertex(max(nodes) + 1)

<generator object <genexpr> at 0x7f46743075f0>

In [115]:
a = 0
for edge in train_set:
    if edge not in poss_set:
        a += 1
        u, w = map(int, edge.split())
        g.add_edge(g.vertex(u), g.vertex(w))
print 1.0 * a / len(train_set)

0.950002183311


In [116]:
is_bi, part = graph_tool.topology.is_bipartite(g, partition=True)

In [117]:
is_bi

True

In [118]:
groups = g.new_vertex_property("int")

In [119]:
for u in g.vertices():
    groups[u] = part[u]

In [120]:
pos_default = sfdp_layout(g)

libgraph_tool_layout.sfdp verbose:  False bipartite:  False
libgraph_tool_layout.sfdp verbose:  False bipartite:  False
libgraph_tool_layout.sfdp verbose:  False bipartite:  False
libgraph_tool_layout.sfdp verbose:  False bipartite:  False
libgraph_tool_layout.sfdp verbose:  False bipartite:  False
libgraph_tool_layout.sfdp verbose:  False bipartite:  False
libgraph_tool_layout.sfdp verbose:  False bipartite:  False
libgraph_tool_layout.sfdp verbose:  False bipartite:  False


In [121]:
pos = sfdp_layout(g, groups=groups, bipartite=False, verbose=True)

Positioning level: 0 26547 with K = 80.2000763544 ...
libgraph_tool_layout.sfdp verbose:  True bipartite:  False
avg edge distance: 367.330431557


In [122]:
pos_bip = sfdp_layout(g, groups=groups, bipartite=True, verbose=True)

Positioning level: 0 26547 with K = 82.4203965723 ...
libgraph_tool_layout.sfdp verbose:  True bipartite:  True
avg edge distance: 137.05928941


In [123]:
pos_bip

<PropertyMap object with key type 'Vertex' and value type 'vector<double>', for Graph 0x7f46800e7b90, at 0x7f46742bcc90>

In [124]:
%matplotlib inline

#graph_draw(g)

In [125]:
%matplotlib inline

#graph_draw(g, pos=pos, vertex_fill_color=groups)

In [126]:
%matplotlib inline

#graph_draw(g, pos=pos_bip, vertex_fill_color=groups)

In [127]:
from sklearn.metrics import roc_auc_score

In [128]:
features = tools.TopologicalFeatures(g, pos_default, gap=0)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.dist])
print "sfdp default: ", roc_auc_score(Y, X)

sfdp default:  0.757684932697


In [129]:
features = tools.TopologicalFeatures(g, pos_bip, gap=0)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.dist])
print "sfdp bipartite simple: ", roc_auc_score(Y, X)

sfdp bipartite simple:  0.847380399984


In [130]:
features = tools.TopologicalFeatures(g, pos)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.dist])
print "sfdp default groups: ",roc_auc_score(Y, X)

sfdp default groups:  0.724813875488


In [131]:
features = tools.TopologicalFeatures(g, pos_bip)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.preferential_attachment])
roc_auc_score(Y, X)

0.84588335250832158

In [132]:
from sklearn.decomposition import NMF
model = NMF(n_components=10, init='random', random_state=0)

matrix = tools.make_sparse_matrix(train_set, nodes, poss_set)

features = tools.MFFeatures(model, matrix)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.score])
roc_auc_score(Y, X)

0.7491702633011913

In [133]:
from scipy.sparse import linalg
import numpy

matrix = tools.make_sparse_matrix(train_set, nodes, poss_set)
U, s, Vh = linalg.svds(matrix.asfptype(), k=30)

def score(u, w):
    return numpy.dot(U[u] * s, Vh.T[w])

features = tools.MFFeatures(model, matrix)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [score])
roc_auc_score(Y, X)

0.69061234953990625